In [ ]:
import pandas as pd

In [ ]:
data = []
with open('../simulation.log', 'r') as fp:
    raw = fp.read()
    for ln in raw.split('\n'):
        data.append(ln)

In [ ]:
df_dict = {
    'timestamp': [],
    'level': [],
    'thread_id': [],
    'process_id': [],
    'description': []
}

for line in data:
    if not line:
        continue
    timestamp, level, thread_id, process_id, description = line.split(' - ', 4)
    df_dict['timestamp'].append(timestamp)
    df_dict['level'].append(level)
    df_dict['thread_id'].append(thread_id)
    df_dict['process_id'].append(process_id)
    df_dict['description'].append(description)

In [ ]:
df = pd.DataFrame(df_dict, index=range(len(df_dict['timestamp'])))

conversation id
- number of total tokens (inputs + outputs)
- time to return the request
- number of turns

In [ ]:
for _, gp in df.groupby('process_id'):
    tmp = gp.sort_values('timestamp')

    

In [ ]:
df.query('process_id == "2472386"').sort_values('timestamp')